In [11]:
import pandas as pd
from lda2vec.nlppipe import Preprocessor

# Data directory
data_dir ="data"
# Where to save preprocessed data
clean_data_dir = "data/clean_data"
# Name of input file. Should be inside of data_dir
input_file = "restaurant.txt"
# Should we load pretrained embeddings from file
load_embeds = True

# Read in data file
df = pd.read_csv(data_dir+"/"+input_file, sep="\n")

# Initialize a preprocessor
P = Preprocessor(df, "texts", max_features=30000, maxlen=10000, min_count=1)

# Run the preprocessing on your dataframe
P.preprocess()

# Load embeddings from file if we choose to do so
if load_embeds:
    # Load embedding matrix from file path - change path to where you saved them
    embedding_matrix = P.load_para("../glove.6B.300d.txt")
else:
    embedding_matrix = None

# Save data to data_dir
P.save_data(clean_data_dir, embedding_matrix=embedding_matrix)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x92 in position 3: invalid start byte

In [4]:
from lda2vec import utils, model
import tensorflow as tf
tf.reset_default_graph()

# Path to preprocessed data
data_path  = "data/clean_data"
# Whether or not to load saved embeddings file
load_embeds = True

# Load data from files
(idx_to_word, word_to_idx, freqs, pivot_ids,
 target_ids, doc_ids, embed_matrix) = utils.load_preprocessed_data(data_path, load_embed_matrix=load_embeds)

# Number of unique documents
num_docs = doc_ids.max() + 1
# Number of unique words in vocabulary (int)
vocab_size = len(freqs)
# Embed layer dimension size
# If not loading embeds, change 128 to whatever size you want.
embed_size = embed_matrix.shape[1] if load_embeds else 128
# Number of topics to cluster into
num_topics = 30
# Amount of iterations over entire dataset
num_epochs = 200
# Batch size - Increase/decrease depending on memory usage
batch_size = 4096
# Epoch that we want to "switch on" LDA loss
switch_loss_epoch = 0
# Pretrained embeddings value
pretrained_embeddings = embed_matrix if load_embeds else None
# If True, save logdir, otherwise don't
save_graph = True


# Initialize the model
m = model(num_docs,
          vocab_size,
          num_topics,
          embedding_size=embed_size,
          pretrained_embeddings=pretrained_embeddings,
          freqs=freqs,
          batch_size = batch_size,
          save_graph_def=save_graph)

# Train the model
m.train(pivot_ids,
        target_ids,
        doc_ids,
        len(pivot_ids),
        num_epochs,
        idx_to_word=idx_to_word,
        switch_loss_epoch=switch_loss_epoch)


W0628 09:35:26.517774  5668 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\lda2vec\Lda2vec.py:40: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0628 09:35:26.517774  5668 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\lda2vec\Lda2vec.py:42: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0628 09:35:26.564133  5668 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\lda2vec\Lda2vec.py:69: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0628 09:35:26.580092  5668 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\lda2vec\word_embedding.py:18: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0628 09:35:26.604027  5668 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\lda2vec\embedding_mixture.py:25: The nam


EPOCH: 1
LOSS 843492.56 w2v 119.21331 lda 843373.4

EPOCH: 2
LOSS 842599.25 w2v 110.51364 lda 842488.75

EPOCH: 3
LOSS 841705.8 w2v 100.64947 lda 841605.2

EPOCH: 4
LOSS 840840.56 w2v 118.58304 lda 840722.0

EPOCH: 5
LOSS 839950.2 w2v 110.818436 lda 839839.4


W0628 09:35:35.821462  5668 deprecation.py:506] From C:\ProgramData\Anaconda3\lib\site-packages\lda2vec\Lda2vec.py:276: calling reduce_sum_v1 (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


---------Closest 10 words to given indexes----------
Topic 0 : requestor, frac, transistor, rf, testability, watt, ipc, hertz, profiler, mbit
Topic 1 : requestor, testability, transistor, imposes, noisy, frac, bureaucracies, consumermark, queued, hartley
Topic 2 : requestor, testability, consumermark, transistor, rf, decompressed, queued, hertz, stimulation, bureaucracies
Topic 3 : requestor, rf, testability, queued, transistor, noisy, mbit, c, frac, ipc
Topic 4 : requestor, testability, consumermark, claude, packets, frac, equivalently, pocket, offs, rf
Topic 5 : requestor, testability, mbit, decompressed, transistor, packets, multipliers, bureaucracies, textstyle, queued
Topic 6 : requestor, watt, c, hartley, queued, testability, rf, ia, consumermark, mbit
Topic 7 : requestor, testability, noisy, frac, imposes, textstyle, mbit, terrestrial, claude, queued
Topic 8 : requestor, imposes, transistor, stimulation, limitations, rf, decompressed, testability, frac, deliverables
Topic 9 : ha

KeyboardInterrupt: 

In [4]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd

def twenty_newsgroup_to_csv():
    newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

    df = pd.DataFrame([newsgroups_train.data, newsgroups_train.target.tolist()]).T
    df.columns = ['text', 'target']

    targets = pd.DataFrame( newsgroups_train.target_names)
    targets.columns=['title']

    out = pd.merge(df, targets, left_on='target', right_index=True)
    out['date'] = pd.to_datetime('now')
    out.to_csv('20_newsgroup.csv')
    
twenty_newsgroup_to_csv()

I0626 21:42:12.768491 15668 twenty_newsgroups.py:247] Downloading 20news dataset. This may take a few minutes.
I0626 21:42:12.769488 15668 twenty_newsgroups.py:80] Downloading dataset from https://ndownloader.figshare.com/files/5975967 (14 MB)
